In [45]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [46]:
# Load the trained model , scaler pickle , onehot
model = load_model('models/model.h5')
# load the encoder and scaaaler
with open('pipelines/one_hot_encoding.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)
with open('pipelines/label_encoder.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)
with open('pipelines/scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [47]:
# Example input data

input_data = {
    'CreditScore': 600,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}

In [48]:
def predict_churn(model, label_encoder, one_hot_encoder, scaler, input_data):

    input_df = pd.DataFrame([input_data])
    geo_one = one_hot_encoder.transform(input_df[['Geography']])
    if hasattr(geo_one, "toarray"):
        geo_one = geo_one.toarray()
    geo_df = pd.DataFrame(geo_one, columns=one_hot_encoder.get_feature_names_out(['Geography']))
    input_df['Gender'] = label_encoder.transform(input_df['Gender'])
    final_df = pd.concat([input_df.drop(['Geography'],axis=1),geo_df],axis=1)

    scaled_df = scaler.transform(final_df)
    prediction = model.predict(scaled_df)
    proba = prediction[0][0]

    return proba

In [49]:
predict_churn(model=model,label_encoder=label_encoder_gender,one_hot_encoder=label_encoder_geo,scaler=scaler,input_data=input_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step


np.float32(0.043509424)